In [5]:
import pandas as pd

# read the CSV file into a pandas DataFrame
df = pd.read_csv('bist_social_media_csv/bistsocial_media_twtsqlite_202304131040.csv')

In [6]:
df = df.drop('about', axis=1)
df = df.drop('account_creation_date', axis=1)
df = df.drop('token', axis=1)
df = df.drop('gender', axis=1)
df = df.drop('image_url', axis=1)
df = df.drop('internal_id', axis=1)
df = df.drop('message_url', axis=1)
df = df.drop('received_timestamp', axis=1)
df = df.drop('retweet_id', axis=1)
df = df.drop('tweets', axis=1)
df = df.drop('hashtag', axis=1)
df = df.drop('language', axis=1)
df = df.drop('location', axis=1)
df = df.drop('platform_type', axis=1)
df = df.drop('message_date', axis=1)
df = df.drop('tweet_id', axis=1)
df = df.drop('date', axis=1)
df = df.drop('followers', axis=1)
df = df.drop('retweet_count', axis=1)

In [7]:
sample_df = df.sample(n=100000)

# write the sample DataFrame to a CSV file
sample_df.to_csv('100000twtunlabeled.csv', index=False)

# confirm that the new file was created and contains the expected data
new_df = pd.read_csv('100000twtunlabeled.csv')
print(new_df.head())

                                      message_detail         username
0  RT @YukselisGruplar: "UÇAK DEDİĞİM DE KÜÇÜK Bİ...       ce34551935
1  @TuNaBeYii Hocam kimden duyduysam bunun için k...       Kasmpaal16
2  RT @SatyaSSRian1: It's high time   SERVE FAIR ...   GunjitBirSingh
3  Ya tamam taraftara göre kadro kurulmaz falan d...  kafalarmatizmat
4  RT @7ameMoo: hedef kitlem sonsuzluk #ankaratra...     shakGltekin3


In [96]:
test_data = pd.read_csv('100000twtfinance.csv', sep=',')

In [207]:
test_data = pd.read_csv('100000twtnotfinance.csv', sep=',')

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score

# Load the labeled training sets and create input tensors
# Assume that the training data is in CSV format with three columns: message_detail, username, and label
train_data_1 = pd.read_csv('100000twtfinance.csv', sep=',')
train_data_2 = pd.read_csv('100000twtnotfinance.csv', sep=',')

train_data_1 = train_data_1.dropna(subset=['label'])
train_data_2 = train_data_2.dropna(subset=['label'])

print(train_data_1.isnull().sum())  # Check for NaN values
print(train_data_1.dtypes)  # Check the data types of each column

print(train_data_2.isnull().sum())  # Check for NaN values
print(train_data_2.dtypes)  # Check the data types of each column

train_texts_1 = train_data_1["message_detail"].tolist()
train_labels_1 = train_data_1["label"].tolist()
train_texts_2 = train_data_2["message_detail"].tolist()
train_labels_2 = train_data_2["label"].tolist()

# Remove missing or invalid labels
train_labels_1 = [label for label in train_labels_1 if label == label] # Removes NaN values
train_labels_2 = [label for label in train_labels_2 if label == label] # Removes NaN values

# Convert labels to integers (if necessary)
train_labels_1 = [int(label) for label in train_labels_1]
train_labels_2 = [int(label) for label in train_labels_2]

tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-turkish-cased')

train_encodings_1 = tokenizer(train_texts_1, truncation=True, padding=True, return_tensors="pt")
train_encodings_2 = tokenizer(train_texts_2, truncation=True, padding=True, return_tensors="pt")

# Check the size of the input tensors
assert train_encodings_1["input_ids"].size(0) == len(train_labels_1), "Size mismatch between input_ids and labels"
assert train_encodings_1["attention_mask"].size(0) == len(train_labels_1), "Size mismatch between attention_mask and labels"
assert train_encodings_2["input_ids"].size(0) == len(train_labels_2), "Size mismatch between input_ids and labels"
assert train_encodings_2["attention_mask"].size(0) == len(train_labels_2), "Size mismatch between attention_mask and labels"

train_dataset_1 = torch.utils.data.TensorDataset(
    train_encodings_1["input_ids"],
    train_encodings_1["attention_mask"],
    torch.tensor(train_labels_1),
)

train_dataset_2 = torch.utils.data.TensorDataset(
    train_encodings_2["input_ids"],
    train_encodings_2["attention_mask"],
    torch.tensor(train_labels_2),
)

# Fine-tune the BERT model on the training sets
model = BertForSequenceClassification.from_pretrained('dbmdz/bert-base-turkish-cased', num_labels=2)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

train_loader_1 = torch.utils.data.DataLoader(train_dataset_1, batch_size=8, shuffle=True)
train_loader_2 = torch.utils.data.DataLoader(train_dataset_2, batch_size=8, shuffle=True)

model.train()
for epoch in range(3):
    for batch in train_loader_1:
        optimizer.zero_grad()
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    for batch in train_loader_2:
        optimizer.zero_grad()
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Load the test set and create input tensors
# Assume that the test data is in CSV format with two columns: message_detail and username
test_data = pd.read_csv('100000twtunlabeled.csv', sep=',')
test_texts = test_data["message_detail"].tolist()

test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors="pt")
test_dataset = torch.utils.data.TensorDataset(
    test_encodings["input_ids"],
    test_encodings["attention_mask"],
)

# Use the trained model to make predictions on the test set
model.eval()
predictions = []
for batch in torch.utils.data.DataLoader(test_dataset, batch_size=8):
    batch = tuple(t.to(device) for t in batch)
    with torch.no_grad():
        inputs = {"input_ids": batch[0], "attention_mask": batch[1]}
        outputs = model(**inputs)
        logits = outputs.logits
        batch_predictions = torch.argmax(logits, axis=1)
        predictions.extend(batch_predictions.detach().cpu().numpy().tolist())

# Print the accuracy of the model
print("Accuracy:", accuracy_score(predictions, [0] * len(predictions)))

message_detail    0
username          0
label             0
dtype: int64
message_detail     object
username           object
label             float64
dtype: object
message_detail    0
username          0
label             0
dtype: int64
message_detail     object
username           object
label             float64
dtype: object


Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were 

In [ ]:
# Use the fine-tuned BERT model to predict the labels of the test set
model.eval()
test_set_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=8)
test_set_predictions = []

for batch in test_set_dataloader:
    batch_input_ids = batch[0].to(device)
    batch_attention_masks = batch[1].to(device)

    with torch.no_grad():
        outputs = model(batch_input_ids, attention_mask=batch_attention_masks)

    logits = outputs[0]
    batch_predictions = logits.argmax(dim=-1).tolist()
    test_set_predictions.extend(batch_predictions)

# Print the Text and the predicted label of each example in the test set
for i, text in enumerate(test_texts):
    label = test_set_predictions[i]
    print(f"{message_detail}\n{label}\n")